In [2]:
import io
import glob
import numpy as np
import pandas as pd
from netCDF4 import Dataset

import dateutil.tz as tz

import pvlib
from pvlib import clearsky, atmosphere, solarposition
from pvlib.location import Location


In [37]:
tus = Location(14.6361, 121.0775, 'Asia/Manila', 100, 'Manila')
times = pd.date_range(start='2019-01-01', end='2020-12-31', freq='1min', tz=tus.tz)

cs = tus.get_solarposition(times)  
cs = cs.reset_index()
cs['Datetime_LT'] = cs['index']

#cs.to_csv('check.csv')

In [29]:

fname = '/Users/apple/Desktop/Others/Python_Codes/Cirrus_Paper/2020_CCDA/Output/2020_01_02_Manila_ODs.nc'
fh = Dataset(fname, mode='r')
fname = fname.split("/")[9].split(".")[0]

ccp = fh['/Cirrus_Cloud_Properties/']
od_mean = pd.DataFrame(ccp.variables['Ice_Cloud_Optical_Depth'][:])
cld_top = pd.DataFrame(ccp.variables['Ice_Cloud_Top_Cloud_Height'][:])
cld_bot = pd.DataFrame(ccp.variables['Ice_Cloud_Bottom_Cloud_Base'][:])
cld_num = pd.DataFrame(ccp.variables['Ice_Cloud_Num'][:])

ts = pd.DataFrame(ccp.variables['time_sec'][:])
tmin = pd.DataFrame(ccp.variables['time_min'][:])
thr = pd.DataFrame(ccp.variables['time_hour'][:])
tday = pd.DataFrame(ccp.variables['time_day'][:])
tmon = pd.DataFrame(ccp.variables['time_month'][:])
ty = pd.DataFrame(ccp.variables['time_year'][:])

df = od_mean 
df['od_mean']  = df[0]
df = df[['od_mean']]
df['cld_top'] = cld_top[0]
df['cld_bot'] = cld_bot[0]
df['cld_num'] = cld_num[0]
df['ts'] = ts[0].astype(int)
df['minute'] = tmin[0].astype(int)
df['hour'] = thr[0].astype(int)
df['day'] = tday[0].astype(int)
df['month'] = tmon[0].astype(int)
df['year'] = ty[0].astype(int)
df['low_cld'] = 'Y'

wat_cld = pd.DataFrame(ccp.variables['Water_Cloud_Backscat_Flag'][:])
wat_cld = wat_cld[0]

for i in range(0, wat_cld.shape[0]-1):
    j = wat_cld.iloc[i].astype(int)
    df['low_cld'].iloc[j] = 'N'
df.to_csv(f'{fname}.csv')

<class 'netCDF4._netCDF4.Group'>
group /Cirrus_Cloud_Properties:
    dimensions(sizes): 
    variables(dimensions): float64 time_year(time), float64 time_month(time), float64 time_day(time), float64 time_hour(time), float64 time_min(time), float64 time_sec(time), float64 Ice_Cloud_Optical_Depth(time), float64 Ice_Cloud_Top_Cloud_Height(time), float64 Ice_Cloud_Bottom_Cloud_Base(time), float64 Ice_Cloud_Num(time), float64 Water_Cloud_Backscat_Flag(idx)
    groups: 


,od_mean,cld_top,cld_bot,cld_num,ts,minute,hour,day,month,year,low_cld
0,NaN,NaN,NaN,NaN,15,1,0,2,1,2020,Y
1,NaN,NaN,NaN,NaN,15,6,0,2,1,2020,Y
2,NaN,NaN,NaN,NaN,15,11,0,2,1,2020,Y
3,NaN,NaN,NaN,NaN,15,16,0,2,1,2020,Y
4,NaN,NaN,NaN,NaN,15,21,0,2,1,2020,Y
...,...,...,...,...,...,...,...,...,...,...,...
284,0.102130,14315.0,12366.0,0.0,46,33,23,2,1,2020,Y
285,NaN,NaN,NaN,0.0,46,38,23,2,1,2020,Y
286,0.080110,14315.0,12366.0,0.0,46,43,23,2,1,2020,Y
287,0.092794,14315.0,12366.0,0.0,46,48,23,2,1,2020,Y


In [31]:

def MNL_HSRL_Ice_Cloud_merge(fname):
    
    fh = Dataset(fname, mode='r')
    fname = fname.split("/")[9].split(".")[0]
    
    ccp = fh['/Cirrus_Cloud_Properties/']
    od_mean = pd.DataFrame(ccp.variables['Ice_Cloud_Optical_Depth'][:])
    cld_top = pd.DataFrame(ccp.variables['Ice_Cloud_Top_Cloud_Height'][:])
    cld_bot = pd.DataFrame(ccp.variables['Ice_Cloud_Bottom_Cloud_Base'][:])
    cld_num = pd.DataFrame(ccp.variables['Ice_Cloud_Num'][:])

    ts = pd.DataFrame(ccp.variables['time_sec'][:])
    tmin = pd.DataFrame(ccp.variables['time_min'][:])
    thr = pd.DataFrame(ccp.variables['time_hour'][:])
    tday = pd.DataFrame(ccp.variables['time_day'][:])
    tmon = pd.DataFrame(ccp.variables['time_month'][:])
    ty = pd.DataFrame(ccp.variables['time_year'][:])

    df = od_mean 
    df['od_mean']  = df[0]
    df = df[['od_mean']]
    df['cld_top'] = cld_top[0]
    df['cld_bot'] = cld_bot[0]
    df['cld_num'] = cld_num[0]
    df['ts'] = ts[0].astype(int)
    df['minute'] = tmin[0].astype(int)
    df['hour'] = thr[0].astype(int)
    df['day'] = tday[0].astype(int)
    df['month'] = tmon[0].astype(int)
    df['year'] = ty[0].astype(int)
    df['low_cld'] = 'Y'

    wat_cld = pd.DataFrame(ccp.variables['Water_Cloud_Backscat_Flag'][:])
    wat_cld = wat_cld[0]

    for i in range(0, wat_cld.shape[0]-1):
        j = wat_cld.iloc[i].astype(int)
        df['low_cld'].iloc[j] = 'N'
    
    #make datetime column for merging with rad data later
    df['Datetime'] = pd.to_datetime(df.year.apply(str) + '-' + df.month.apply(str) + '-' + df.day.apply(str) + ' ' + df.hour.apply(str) + ':' + df.minute.apply(str), format='%Y-%m-%d %H:%M')
    df['Datetime'] = df['Datetime'].dt.tz_localize('UTC')
    df['Datetime_LT'] = df['Datetime'].dt.tz_convert('Asia/Manila')

    df['month_LT'] = pd.DatetimeIndex(df['Datetime_LT']).month
    df['day_LT'] = pd.DatetimeIndex(df['Datetime_LT']).day
    df['hour_LT'] = pd.DatetimeIndex(df['Datetime_LT']).hour
    df['u_day'] = pd.to_datetime(df.year.apply(str) + '-' + df.month_LT.apply(str) + '-' + df.day_LT.apply(str) )
    loc = 'csv/input/'
    df.to_csv(f'{loc}{fname}_ods.csv')
    
    return


In [43]:
yyyy = 2019
loc = 'csv/input/'
all_files = glob.glob(f"/Users/apple/Desktop/Others/Python_Codes/Cirrus_Paper/2020_CCDA/Output/*{yyyy}*nc")

for filename in all_files:
    fname = filename
    MNL_HSRL_Ice_Cloud_merge(fname)

glued_data = pd.DataFrame()


for file_name in glob.glob(f'{loc}*ods.csv'):
    x = pd.read_csv(file_name, low_memory=False)
    glued_data = pd.concat([glued_data,x],axis=0)

glued_data.to_csv(f'csv/output/{yyyy}_ODs.csv')
df = glued_data

In [44]:
'''
glued_data = pd.DataFrame()

loc = 'csv/2019/input/'
for file_name in glob.glob(f'{loc}*ods.csv'):
    x = pd.read_csv(file_name, low_memory=False)
    #print(x)
    glued_data = pd.concat([glued_data,x],axis=0)

    
df = glued_data'''

"\nglued_data = pd.DataFrame()\n\nloc = 'csv/2019/input/'\nfor file_name in glob.glob(f'{loc}*ods.csv'):\n    x = pd.read_csv(file_name, low_memory=False)\n    #print(x)\n    glued_data = pd.concat([glued_data,x],axis=0)\n\n    \ndf = glued_data"

In [45]:
#make datetime column for merging with rad data later
df['Datetime'] = pd.to_datetime(df.year.apply(str) + '-' + df.month.apply(str) + '-' + df.day.apply(str) + ' ' + df.hour.apply(str) + ':' + df.minute.apply(str), format='%Y-%m-%d %H:%M')
df['Datetime'] = df['Datetime'].dt.tz_localize('UTC')


In [46]:
df['Datetime_LT'] = df['Datetime'].dt.tz_convert('Asia/Manila')

df['month_LT'] = pd.DatetimeIndex(df['Datetime_LT']).month
df['day_LT'] = pd.DatetimeIndex(df['Datetime_LT']).day
df['hour_LT'] = pd.DatetimeIndex(df['Datetime_LT']).hour

df['u_day'] = pd.to_datetime(df.year.apply(str) + '-' + df.month_LT.apply(str) + '-' + df.day_LT.apply(str) )

In [47]:
df = df.merge(cs, how='inner', on='Datetime_LT')

In [48]:
df.loc[df['month'] <= 2, 'season'] = 'DJF'
df.loc[(df['month'] >= 3) & (df['month'] <= 5), 'season'] = 'MAM'
df.loc[(df['month'] >= 6) & (df['month'] <= 8), 'season'] = 'JJA'
df.loc[(df['month'] >= 9) & (df['month'] <= 11), 'season'] = 'SON'
df.loc[df['month'] == 12, 'season'] = 'DJF'

In [49]:
#group by time
df.loc[(df['apparent_zenith'] >= 90), 'Period'] = 'Night' 
df.loc[(df['apparent_zenith'] < 90), 'Period'] = 'Day'


In [50]:
df.to_csv(f'{yyyy}_hsrl_ods_all.csv')

In [51]:
df_od = df[df['cld_bot'] >= 0]
df_od.to_csv(f'{yyyy}_hsrl_ods_cirrus_only.csv')